<a href="https://colab.research.google.com/github/Niroth36/Text_Classification_Natural_Language_Processing/blob/main/Text_Classification_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [21]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

from keras import layers
from keras import losses
from keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Importing dataset

Download the CSV file

In [34]:
url = "https://raw.githubusercontent.com/Niroth36/Text_Classification_Natural_Language_Processing/main/dataset.csv.gz"
filename = "dataset.csv.gz"

path = tf.keras.utils.get_file(filename, url)

Extract the file and read the CSV file using pandas

In [35]:
df = pd.read_csv(path, compression='gzip')

In [41]:
df.head()

,Unnamed: 0,Release Year,Title,Origin/Ethnicity,Director,Wiki Page,Plot,Genre
0,6,1903,The Great Train Robbery,American,Edwin S. Porter,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,action
1,7,1904,The Suburbanite,American,Wallace McCutcheon,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,comedy
2,13,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,biography
3,14,1907,How Brown Saw the Baseball Game,American,Unknown,https://en.wikipedia.org/wiki/How_Brown_Saw_th...,Before heading out to a baseball game at a nea...,comedy
4,15,1907,Laughing Gas,American,Edwin Stanton Porter,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...,comedy


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23025 entries, 0 to 23024
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        23025 non-null  int64 
 1   Release Year      23025 non-null  int64 
 2   Title             23025 non-null  object
 3   Origin/Ethnicity  23025 non-null  object
 4   Director          23025 non-null  object
 5   Cast              22539 non-null  object
 6   Wiki Page         23025 non-null  object
 7   Plot              23025 non-null  object
 8   Genre             23025 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.6+ MB


In [40]:
df.isnull().sum()

Unnamed: 0          0
Release Year        0
Title               0
Origin/Ethnicity    0
Director            0
Wiki Page           0
Plot                0
Genre               0
dtype: int64

I decided to drop Cast column because it has missing values and it has no point in categorizing the movies according to their genre.

In [39]:
df= df.drop("Cast", axis='columns')

This function should load the data from the CSV file at file_path, preprocess it as described, and return three tf.data.Dataset objects for training, validation, and testing, respectively. The batch_size, p_train, and p_val parameters control the size of the batches and the proportion of the data used for training, validation, and testing, respectively. The function also returns the number of labels in the dataset as n_labels.

In [14]:
def load_data_wiki(file_path, batch_size=32, p_train=.65, p_val=0.15):
    # load data with pandas
    df = pd.read_csv(file_path)

    # randomize the data
    df = df.sample(frac=1, random_state=42)

    # convert Origin/Ethnicity to one-hot encoding
    ethnicity_onehot = pd.get_dummies(df['Origin/Ethnicity'], prefix='Ethnicity')

    # concatenate the data and one-hot encoded ethnicity
    df = pd.concat([df, ethnicity_onehot], axis=1)

    # convert the genre labels to integer IDs
    genre2id = {g: i for i, g in enumerate(sorted(df['Genre'].unique()))}
    df['label'] = df['Genre'].map(genre2id)

    # split data into train, val, and test
    train_size = int(p_train * len(df))
    val_size = int(p_val * len(df))
    test_size = len(df) - train_size - val_size

    train_df = df[:train_size]
    val_df = df[train_size:train_size + val_size]
    test_df = df[train_size + val_size:]

    # convert dataframes to dictionaries
    train_data = dict(train_df[['Title', 'Plot', 'Ethnicity_American', 'Ethnicity_British', 'Ethnicity_Indian', 'Ethnicity_Other']].items())
    train_data['label'] = np.array(train_df['label'].tolist())

    val_data = dict(val_df[['Title', 'Plot', 'Ethnicity_American', 'Ethnicity_British', 'Ethnicity_Indian', 'Ethnicity_Other']].items())
    val_data['label'] = np.array(val_df['label'].tolist())

    test_data = dict(test_df[['Title', 'Plot', 'Ethnicity_American', 'Ethnicity_British', 'Ethnicity_Indian', 'Ethnicity_Other']].items())
    test_data['label'] = np.array(test_df['label'].tolist())

    # create tf.data.Dataset objects
    train_ds = tf.data.Dataset.from_tensor_slices(train_data)
    train_ds = train_ds.shuffle(buffer_size=train_size)
    train_ds = train_ds.batch(batch_size)

    val_ds = tf.data.Dataset.from_tensor_slices(val_data)
    val_ds = val_ds.shuffle(buffer_size=val_size)
    val_ds = val_ds.batch(batch_size)

    test_ds = tf.data.Dataset.from_tensor_slices(test_data)
    test_ds = test_ds.batch(batch_size)

    # return datasets and number of labels
    return train_ds, val_ds, test_ds, len(genre2id)


In [16]:
import string
import re

def preprocess_titles(title):
    # Convert to lowercase
    title = title.lower()
    # Remove punctuation
    title = title.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    title = re.sub(r'\d+', '', title)
    # Split into words
    title_words = title.split()
    # Remove stop words
    stop_words = set(['a', 'an', 'and', 'the', 'of', 'in', 'on', 'at', 'is'])
    title_words = [word for word in title_words if word not in stop_words]
    # Return preprocessed title as string
    return ' '.join(title_words)


In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Load the movie plots dataset
df = pd.read_csv(path, compression='gzip')

# Preprocess the titles column
def preprocess_titles(title):
    # Remove parentheses and their contents
    title = re.sub(r'\([^)]*\)', '', title)
    # Remove any remaining punctuation
    title = re.sub(r'[^\w\s]', '', title)
    # Convert to lowercase
    title = title.lower()
    return title

df['Title'] = df['Title'].apply(preprocess_titles)

# Convert Origin/Ethnicity and Genre to one-hot encoded variables
df = pd.get_dummies(df, columns=['Origin/Ethnicity', 'Genre'])

# Split the dataset into train, validation, and test sets
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=42), [int(0.6*len(df)), int(0.8*len(df))])

# Convert the dataframes to dictionaries
train_dict = {'Title': train_df['Title'].values, 'Origin/Ethnicity': train_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': train_df['Plot'].values, 'Genre': train_df.filter(like='Genre').values.astype(np.float64)}
val_dict = {'Title': val_df['Title'].values, 'Origin/Ethnicity': val_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': val_df['Plot'].values, 'Genre': val_df.filter(like='Genre').values.astype(np.float64)}
test_dict = {'Title': test_df['Title'].values, 'Origin/Ethnicity': test_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': test_df['Plot'].values, 'Genre': test_df.filter(like='Genre').values.astype(np.float64)}

# Create tf.data.Dataset objects from the dictionaries
train_dataset = tf.data.Dataset.from_tensor_slices(train_dict)
val_dataset = tf.data.Dataset.from_tensor_slices(val_dict)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dict)

# Print the first few entries in the training dataset
for elem in train_dataset.take(5):
    print(elem)


{'Title': <tf.Tensor: shape=(), dtype=string, numpy=b'creature with the atom brain'>, 'Origin/Ethnicity': <tf.Tensor: shape=(24,), dtype=float64, numpy=
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])>, 'Plot': <tf.Tensor: shape=(), dtype=string, numpy=b"A hulking zombie breaks into a mansion and kills a gangster named Hennesy. The blood stains left behind at the crime scene are radioactive, and the fingerprints of the killer are of a man who had died days before the murder; the police are baffled.\r\nGangster boss Frank Buchanan, who had been forced to flee the United States before he was deported, was betrayed by members of his own underworld gang. While traveling in Europe, he finds ex-Nazi scientist Wilhelm Steigg (Gaye), who is trying to reanimate the dead in order to provide a menial labor pool that is easily exploited. Buchanan funds the research and brings the scientist to America with the unstated goal of sending S

The dictionary can now be used with tf.data.Dataset.from_tensor_slices to create the training dataset. The same process can be followed for the validation and test sets. Be sure to print intermediate results (dataframes, datasets) to ensure that the datasets are in the expected format before using them for training.

In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Load the movie plots dataset
df = pd.read_csv(path, compression='gzip')

# Preprocess the titles column
def preprocess_titles(title):
    # Remove parentheses and their contents
    title = re.sub(r'\([^)]*\)', '', title)
    # Remove any remaining punctuation
    title = re.sub(r'[^\w\s]', '', title)
    # Convert to lowercase
    title = title.lower()
    return title

df['Title'] = df['Title'].apply(preprocess_titles)

# Convert Origin/Ethnicity and Genre to one-hot encoded variables
df = pd.get_dummies(df, columns=['Origin/Ethnicity', 'Genre'])

# Split the dataset into train, validation, and test sets
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=42), [int(0.6*len(df)), int(0.8*len(df))])

# Convert the dataframes to dictionaries
train_dict = {'Title': train_df['Title'].values, 'Origin/Ethnicity': train_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': train_df['Plot'].values, 'Genre': train_df.filter(like='Genre').values.astype(np.float64)}
val_dict = {'Title': val_df['Title'].values, 'Origin/Ethnicity': val_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': val_df['Plot'].values, 'Genre': val_df.filter(like='Genre').values.astype(np.float64)}
test_dict = {'Title': test_df['Title'].values, 'Origin/Ethnicity': test_df.filter(like='Origin/Ethnicity').values.astype(np.float64), 'Plot': test_df['Plot'].values, 'Genre': test_df.filter(like='Genre').values.astype(np.float64)}

# Create tf.data.Dataset objects from the dictionaries
train_dataset = tf.data.Dataset.from_tensor_slices(train_dict)
val_dataset = tf.data.Dataset.from_tensor_slices(val_dict)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dict)

# Print the first few entries in the training dataset
for elem in train_dataset.take(5):
    print(elem)


{'Title': <tf.Tensor: shape=(), dtype=string, numpy=b'creature with the atom brain'>, 'Origin/Ethnicity': <tf.Tensor: shape=(24,), dtype=float64, numpy=
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])>, 'Plot': <tf.Tensor: shape=(), dtype=string, numpy=b"A hulking zombie breaks into a mansion and kills a gangster named Hennesy. The blood stains left behind at the crime scene are radioactive, and the fingerprints of the killer are of a man who had died days before the murder; the police are baffled.\r\nGangster boss Frank Buchanan, who had been forced to flee the United States before he was deported, was betrayed by members of his own underworld gang. While traveling in Europe, he finds ex-Nazi scientist Wilhelm Steigg (Gaye), who is trying to reanimate the dead in order to provide a menial labor pool that is easily exploited. Buchanan funds the research and brings the scientist to America with the unstated goal of sending S

In this example, preprocess_titles() is a function that extracts the movie titles from the text data, and y is the target variable for classification (e.g., 1 for action movies, 0 for romance movies). The number of epochs is set to 50, but this value can be adjusted based on the performance of the model on the training data.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.feature_extraction.text import CountVectorizer

# Preprocess the text data to extract only the movie title
titles = preprocess_titles("The Shawshank Redemption")

# Convert the titles into a binary bag-of-words representation with a vocabulary of 500 words
vectorizer = CountVectorizer(binary=True, max_features=500)
X = vectorizer.fit_transform(titles)

# Create a linear classification model with a single output layer
model = Sequential()
model.add(Dense(1, input_dim=X.shape[1], activation='sigmoid'))

# Compile the model using the Adam optimizer with a learning rate of 0.001 and the binary cross-entropy loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the fit method of the Keras model
model.fit(X, y, epochs=50, batch_size=32)
